### Gathering Data from Web

Idea is to use python to extract the data from Web and use it for analysis


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/Datahack\ NLP\ Workshop

 big.txt	    Disaster   HelperCodes    Water
'Colab Notebooks'   GMB_NER    HotelReviews


In [0]:
import time
import requests
from datetime import timedelta, date

### Scrape the data from URL

In [0]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

url = "http://123.63.203.150/reserve.asp"
data_path = "/content/drive/My Drive/Datahack NLP Workshop/Water/"
start_dt = date(2016, 1, 1)
end_dt = date(2016, 1, 3) 

for dt in daterange(start_dt, end_dt):
    # post the information and get the html response
    ldate = str(dt.strftime("%d/%m/%Y"))
    data = {"ldate":ldate, "b1":"submit"}
    html_response = requests.post(url, data)
    
    # save it to file
    fname = str(dt.strftime("%d%m%Y")) + ".html"
    with open(data_path+fname, "wb") as f:
        f.write(html_response.content)

### Parse the HTML file

In [0]:
import re
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import timedelta, date

In [0]:
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)

In [0]:
### Get Water Level Data
df = []
for dt in daterange(start_dt, end_dt):
        fname = data_path + str(dt.strftime("%d%m%Y")) + ".html"
        with open(fname, "rb") as fhandle:
                html_doc = fhandle.read()

        soup = BeautifulSoup(html_doc, 'html.parser')
        text_list = []
        for td in soup.find_all("td"):
                text = " ".join(td.get_text().strip().split())
                text_list.append(text)
        text_list = text_list[:-3]

        try:
                text_list = np.array(text_list).reshape([10,9])
        except:
                print(fname)
                print(text_list)
                print(prev_text_list)
                print()
                text_list = prev_text_list

        olist = [str(dt.strftime("%d-%m-%Y")), text_list[1,4], text_list[2,4], text_list[3,4], text_list[5,4]]
        prev_text_list = text_list[:]
        df.append(olist)

df = pd.DataFrame(df, columns=["Date", "POONDI", "CHOLAVARAM", "REDHILLS", "CHEMBARAMBAKKAM"])
df.to_csv(data_path + "chennai_reservoir_levels.csv", index=False)